In [1]:
#!/usr/bin/python3
import numpy as np
import random
import json
import time
import csv
from util import *
import numpy as np
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
from sklearn import svm
import nltk
from joblib import dump, load
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from nltk.tokenize import TweetTokenizer
from sklearn.neural_network import MLPClassifier
import sys, os, re
lex = liwc.parse_liwc("2015")

# IS
advice = ['advice', 'option', 'tip', 'suggestion']
trial = ['try', 'tried', 'I\'ve', 'trying', 'therapist', 'med', 'therapy', 'doctor', 'pill']
question = ['?']
unknow = ['what to do', 'I don\'t know', 'not sure', 'what else to do', 'what\'s the point']
need_help_IS = ['need help', 'any of you', 'anyone else', 'need support', 'feel the same', 'any feedback', 'help me']
negative_words_IS = ['suicide', 'suicidal', 'cut myself', 'hurt myself', 'die', 'dying', 'dead', 'death', 
                  'can\'t leave', 'self harm', 'kill', 'no hope']

# ES
lonely = ['lonely', 'alone', 'loneliness', 'no friend', 'no one', '0 friend', 'No friend', 
         'nobody', 'no fucking friend', 'dont have friend', 'dont have any friend', 'don\'t have any friend']
sadness = ['sad', 'tired', 'horrible', 'terrified', 'scared', 'afraid', 'frustrated', 'awful',
                'pathetic', 'upset', 'fear', 'boring', 'Boring', 'miserable', 'tiring', 'feel down']
want_a_talk = ['want to talk', 'talk to me', 'talks to me', 'Anyone else the same', 'Does anyone', 'Do anyone', 'needed to get it out', 
              'does everyone else', 'Does everyone else', 'need help', 'anyone else', 'any of you', 'feel the same',
              'in the same situation', 'someone can relate', 'Anybody else', 'anybody else', 'someone to talk to', 'Has everyone ever felt', 
              'Is anyone else', 'someone please pray']
feeling = ['feel']
negative_words_ES = ['suicide', 'suicidal', 'cut myself', 'hurt myself', 'die', 'dying', 'dead', 'death', 
                  'can\'t leave', 'self-harm', 'kill myself', 'no hope', 'hopeless', 'don\'t want to live',
                 'not worth being alive', 'kills me', 'kill me', 'cutting myself', 'worthless', 'killing me', 'kill yourself',
                 'meaningless', 'self harm', 'end my life', 'meaningless']
            
def get_vector_post(doc, cats_IS, cats_ES):
    dic = extract(lex, doc)
    # First add the LIWC features
    vec_IS = np.zeros(len(cats_IS))
    for i in range(len(cats_IS)):
        if cats_IS[i] in dic.keys():
            vec_IS[i] = dic[cats_IS[i]]
    vec_ES = np.zeros(len(cats_ES))
    for i in range(len(cats_ES)):
        if cats_ES[i] in dic.keys():
            vec_ES[i] = dic[cats_ES[i]]

# Feature of sentence count, weithed by 8 in IS model, weighted by 9 in ES model, based on trial and error
    sentences = sent_tokenize(doc)
    number_of_sentences = len(sentences)
            
# Feature of words count, using twitter curpons, weighted by 180 in IS model, weighted by 150 in ES model, based on trial and error
    tknzr = TweetTokenizer()
    words = tknzr.tokenize(doc)
    number_of_words = len(words)

# tags feature
    pos = nltk.pos_tag(words)
    total_pos = 0
    proper_nouns = 0
    adj = 0
    cardinal_numbers = 0
    for word, tag in pos:
        total_pos += 1
        if tag == 'NNP' or tag == 'NNPS':
            proper_nouns += 1
        elif tag == 'JJ' or tag == 'JJR' or tag == 'JJS':
            adj += 1
        elif tag =='CD':
            cardinal_numbers += 1
    
# Features for IS
    # Sentence weighted by 8
    if number_of_sentences > 8:
        vec_IS = np.append(vec_IS, [1])
    else:
        sentence_rate = number_of_sentences / 8
        vec_IS = np.append(vec_IS, [sentence_rate])
            
    # Number of words weighted by 180   
    if number_of_words > 180:
        vec_IS = np.append(vec_IS, [1])
    else:
        word_rate = number_of_words / 180  
        vec_IS = np.append(vec_IS, [word_rate])
        
    # Feature of url
    url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', doc)            
    if len(url) > 0:
        vec_IS = np.append(vec_IS, [1])
    else:
        vec_IS = np.append(vec_IS, [0])
        
    # POS tags
    vec_IS = np.append(vec_IS, [proper_nouns / total_pos, adj / total_pos, cardinal_numbers / total_pos])
    
    # advice
    num_words = 0
    for i in advice:
        j = doc.count(i)
        num_words += j
    rate_1 = num_words / number_of_words
    vec_IS = np.append(vec_IS, [rate_1])  
        
    # need help_IS
    num_words = 0
    for i in need_help_IS: 
        j = doc.count(i)
        num_words += j
    rate_1 = num_words / number_of_words
    vec_IS = np.append(vec_IS, [rate_1])   

    # unknow
    num_words = 0
    for i in unknow:
        j = doc.count(i)
        num_words += j
    rate_1 = num_words / number_of_words
    vec_IS = np.append(vec_IS, [rate_1])  

    # trial
    num_words = 0
    for i in trial:
        j = doc.count(i)
        num_words += j
    rate_1 = num_words / number_of_words
    vec_IS = np.append(vec_IS, [rate_1])  

    # question # implementation based on '?'
    num_words = 0
    for i in question:
        j = doc.count(i)
        num_words += j
    if num_words > 4:
        rate_1 = 1
    else:
        rate_1 = num_words / 4
    vec_IS = np.append(vec_IS, [rate_1])  

    # negative_words_IS
    neg_words = 0
    for i in negative_words_IS:
        j = doc.count(i)
        neg_words += j
    rate_1 = neg_words / 4
    vec_IS = np.append(vec_IS, [rate_1])
####################################################################################
# feature for ES
    # Sentence weighted by 9
    if number_of_sentences > 9:
        vec_ES = np.append(vec_ES, [1])
    else:
        sentence_rate = number_of_sentences / 9
        vec_ES = np.append(vec_ES, [sentence_rate])
            
    # Number of words weighted by 150   
    if number_of_words > 150:
        vec_ES = np.append(vec_ES, [1])
    else:
        word_rate = number_of_words / 150
        vec_ES = np.append(vec_ES, [word_rate])
        
    # POS tags
    vec_ES = np.append(vec_ES, [proper_nouns / total_pos, adj / total_pos, cardinal_numbers / total_pos])
        
    # lonely
    num_words = 0
    for i in lonely:
        if doc.count(i) > 0:
            num_words = 1
            break
    vec_ES = np.append(vec_ES, [num_words])

    # sadness
    num_words = 0
    for i in sadness:
        if doc.count(i) > 0:
            num_words = 1
            break
    vec_ES = np.append(vec_ES, [num_words]) 
        
    # want_a_talk
    num_words = 0
    for i in want_a_talk:
        if doc.count(i) > 0:
            num_words = 1
            break
    vec_ES = np.append(vec_ES, [num_words]) 

    # feeling
    num_words = 0
    for i in feeling:
        j = doc.count(i)
        num_words += j
    if num_words > 4:
        rate_1 = 1
    else:
        rate_1 = num_words / 4
    vec_ES = np.append(vec_ES, [rate_1])  
        
    # negative_words_ES
    neg_words = 0
    for i in negative_words_ES:
        if doc.count(i) > 0:
            neg_words = 1
            break
    vec_ES = np.append(vec_ES, [neg_words])
        
    return vec_IS, vec_ES

LIWC_FILES = {
  "2007": "LIWC/LIWC2007_English100131.dic",
  "2015": "LIWC/LIWC2015_English.dic",
}
CAT_DELIM = "%"
f = open(LIWC_FILES["2015"])
LIWC_all = [] # This stores all the 73 LIWC features
cats_section = False
for l in f:
    l = l.strip()
    if l == CAT_DELIM: 
        cats_section = not cats_section
        continue
    if cats_section:
        try:
            i, cat = l.split("\t")
            cat = cat.split()[0]
            LIWC_all.append(cat)
        except: pass # likely hierarchical category tags
    else:
        break
        
black_IS_1 = ['money', 'relig', 'death', 'netspeak', 'assent', 'nonflu', 'anx', 'family'] 
black_ES_1 = ['male', 'ingest', 'body', 'number', 'filler', 'money', 'shehe', 'they']
IS_1 = []
ES_1 = []
for i in LIWC_all:
    if i not in black_IS_1:
        IS_1.append(i)
for i in LIWC_all:
    if i not in black_ES_1:
        ES_1.append(i)
        
# 提取comment的feature的函数
def get_vector_comment(doc, cats):
    dic = extract(lex, doc)
    vec = np.zeros(len(cats))
    for i in range(len(cats)):
        if cats[i] in dic.keys():
            vec[i] = dic[cats[i]]
    url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', doc)
    if len(url) > 0:
        vec = np.append(vec, [1])
    else:
        vec = np.append(vec, [0])
    sentences = sent_tokenize(doc)
    number_of_sentences = len(sentences)
    if number_of_sentences > 10:
        vec = np.append(vec, [1])
    else:
        sentence_rate = number_of_sentences / 10
        vec = np.append(vec, [sentence_rate])      
    tknzr = TweetTokenizer()
    words = tknzr.tokenize(doc)
    number_of_words = len(words)
    if number_of_words > 60:
        vec = np.append(vec, [1])
    else:
        word_rate = number_of_words / 60
        vec = np.append(vec, [word_rate])
    return vec

comment_LIWC = ['function', 'pronoun', 'ppron', 'i', 'we', 'you', 'shehe', 'they', 
'ipron', 'article', 'prep', 'auxverb', 'adverb', 'conj', 'negate', 'verb', 'adj', 
'compare', 'interrog', 'number', 'quant', 'affect', 'posemo', 'negemo', 'anger', 
'sad', 'social', 'friend', 'female', 'male', 'cogproc', 'insight', 'cause', 
'discrep', 'tentat', 'certain', 'differ', 'percept', 'see', 'hear', 'feel', 
'bio', 'body', 'health', 'sexual', 'ingest', 'drives', 'affiliation', 'achieve', 
'power', 'reward', 'risk', 'focuspast', 'focuspresent', 'focusfuture', 'relativ', 
'motion', 'space', 'time', 'work', 'leisure', 'home', 'informal', 'swear', 'filler']

print('Finishing work preparation!')

Finishing work preparation!


In [6]:
# Loading models
post_IS_clf = load('models/IS_model_post.joblib')
post_ES_clf = load('models/ES_model_post.joblib')
comment_IS_clf = load('models/IS_model_comment.joblib') 
comment_ES_clf = load('models/ES_model_comment.joblib')
# Predict sought IS and ES in post
# Example post:
example_post = "Hi everyone... Last time I spoke to my therapist was on February 3, and I explained to her how my thoughts were getting worse. She referred me to a psychiatrist to discuss antidepressants, though I'm kind of nervous. We talked briefly about what to expect, and she said it'd be more clinical than a therapy session might be. But it wasn't in depth, I was wondering what I could expect from a psychiatric intake, and what (if anything) I should think about to prepare. I'm kind of nervous about it because I don't know what to expect and I'm still not sure how to feel about antidepressants. But I'm still going to go, so I want to know what I can expect. I'm not sure who else to ask. Thanks for reading :)"
post_features_IS, post_features_ES = get_vector_post(example_post, IS_1, ES_1)
post_IS_score = post_IS_clf.predict(post_features_IS.reshape(1, -1))
post_ES_score = post_ES_clf.predict(post_features_ES.reshape(1, -1))
print('The amount (1 - low, 2 - medium, 3 -high) of sought informational support (IS) predicted in the example post is: ')
print(post_IS_score)
print('The amount (1 - low, 2 - medium, 3 -high) of sought emotional support (ES) predicted in the example post is: ')
print(post_ES_score)                   
                      
# Predict received IS and ES in comment
example_comment = "It's not like a therapy session. If you remember your first meeting with your therapist, it'll probably be like that. They ask a lot of questions about you in general, your psychiatric history, your symptoms, etc. It's generally quite a long appointment, since they want to be thorough. You don't have to prepare anything, just be ready to answer a lot of questions. Toward the end they'll probably talk about medication options."
comment_features = get_vector_comment(example_comment, comment_LIWC)
comment_IS_score = comment_IS_clf.predict(comment_features.reshape(1, -1))
comment_ES_score = comment_ES_clf.predict(comment_features.reshape(1, -1))
print('The amount (1 - low, 2 - medium, 3 -high) of provided informational support (IS) predicted in the example comment is: ')
print(comment_IS_score)
print('The amount (1 - low, 2 - medium, 3 -high\) of provided emotional support (ES) predicted in the example comment is: ')
print(comment_ES_score)                   

The amount (1 - low, 2 - medium, 3 -high) of sought informational support (IS) predicted in the example post is: 
[3]
The amount (1 - low, 2 - medium, 3 -high) of sought emotional support (ES) predicted in the example post is: 
[1]
The amount (1 - low, 2 - medium, 3 -high) of provided informational support (IS) predicted in the example comment is: 
[3]
The amount (1 - low, 2 - medium, 3 -high\) of provided emotional support (ES) predicted in the example comment is: 
[2]
